In [29]:
import numpy as np
import torch
import os
from glob import glob
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from tqdm.notebook import tqdm
import torch.nn as nn
import albumentations as A


import logging
import os
import sys
import tempfile
from glob import glob

import torch
from PIL import Image
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
# from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
)
from monai.visualize import plot_2d_or_3d_image
from model_seg_ravir_net import segRAVIR

In [30]:
path2train_images = Path("/scratch/scratch6/akansh12/ravir_data/RAVIR Dataset/train/training_images/")
path2train_mask = Path("/scratch/scratch6/akansh12/ravir_data/ravir/data/training_images_masks/")
path2test_images = Path("/scratch/scratch6/akansh12/ravir_data/RAVIR Dataset/test/")

In [31]:
class ravir_dataset():
    def __init__(self, img_dir, mask_dir, transform = None, data_type = 'train'):
        self.segs_dir = None
        if data_type == 'train':
            self.images_dir = sorted(glob(os.path.join(path2train_images, "*.png")))[:19]
            self.segs_dir = sorted(glob(os.path.join(path2train_mask, "*.npy")))[:19]
        if data_type == "val":
            self.images_dir = sorted(glob(os.path.join(path2train_images, "*.png")))[19:]
            self.segs_dir = sorted(glob(os.path.join(path2train_mask, "*.npy")))[19:]
        self.transform = transform
        
    def __len__(self):
        return len(self.images_dir)
    
    def __getitem__(self, idx):
        img = np.array(Image.open(self.images_dir[idx])).astype('uint8')
        
        if self.segs_dir is not None:
            seg = np.load(self.segs_dir[idx]).astype('uint8')
            img = self.transform(img)
            return img, torch.FloatTensor(seg)
        else:
            img = self.transform(img)
            return img

In [32]:
train_dataset = ravir_dataset(path2train_images, path2train_mask, transform=transforms.ToTensor(), data_type='train')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)

In [34]:
for x,y in train_dataset:
    print(x.shape)
    print(y.shape)
    break

In [11]:
x = torch.randn((4,1,128,128))

In [12]:
x.shape

torch.Size([4, 1, 128, 128])

In [13]:
model = segRAVIR()

In [14]:
y_hat, x_hat = model(x)

In [15]:
y_hat.shape

torch.Size([4, 3, 128, 128])

In [20]:
import monai

ModuleNotFoundError: No module named 'monai'